In [18]:
from typing import List, Set
from enum import Enum
from random import randint
from itertools import combinations_with_replacement

In [21]:
from typing import Tuple


class Partition:
    def __init__(self, xLeft: float, xRight: float, leftNodes: List[Node], rightNodes: List[Node], zeroNode: Node, lattice: AcNetworkLattice, graph: CircuitGraph) -> None:
        self.xLeft = round(xLeft, 3)
        self.xRight = round(xRight, 3)
        self.leftNodes = leftNodes
        self.rightNodes = rightNodes
        self.zeroNode: Node = zeroNode
        self.firstCell = None
        self.lastCell = None
        self.__payloads: List[Payload] = []
        self.__cellQty = 1
        self.lattice = lattice
        self.graph = graph

    def ensureCapacity(self, payloadCoordinates: List[float]):
        coordinates: Set[float] = set()
        for x in payloadCoordinates:
            if gt(x, self.xLeft) and ls(x, self.xRight) and not eq(x, self.xLeft) and not eq(x, self.xRight):
                coordinates.add(round(x, 3))
        print(coordinates)
        if len(coordinates) > self.__cellQty:
            self.__cellQty = len(coordinates) + 1

    def initCells(self) -> None:
        self.firstCell = Cell(self.xLeft, self.xRight, self.leftNodes, self.rightNodes, self.zeroNode, "1", self.lattice, self.graph)
        prev = self.firstCell
        for i in range(1, self.__cellQty):
            rightNodes = [Node(self.xRight, n.trackNumber) for n in self.rightNodes]
            next = Cell(self.xRight, self.xRight, prev.rightNodes, rightNodes, self.zeroNode, str(i + 1), self.lattice, self.graph)
            next.prev = prev
            prev.next = next
            prev = next
        self.lastCell = prev

    def addPayload(self, pl:Payload) -> bool:
        if self.firstCell is None or self.lastCell is None:
            raise Exception()
        if gt(pl.x, self.xLeft) and ls(pl.x, self.xRight):
            self.__payloads.append(pl)
            return True
        return False

    def addPayloads(self, pls: List[Payload]) -> int:
        cnt = 0
        for pl in pls:
            if self.addPayload(pl):
                cnt += 1
        return cnt

    def arrangePayloads(self) -> None:
        if self.firstCell is None or self.lastCell is None:
            raise Exception()
        self.__payloads.sort(key=lambda pl: pl.x)
        self.firstCell.pullRightSection(self.lastCell.xRight) # стянуть все ячейки к правой границе
        cell = self.firstCell
        for pl in self.__payloads:
            cell, n = self.__addPayloadToCell(cell, pl)
            self.graph.addEdge(n, self.zeroNode, pl.iplEdge)
    
    def removePayloads(self):
        self.__payloads.clear()   

    def __addPayloadToCell(self, cell: Cell, pl: Payload) -> Tuple[Cell, Node]:
        if eq(cell.xLeft, pl.x) or eq(cell.xRight, pl.x):
            return cell, cell.findNodeToConnect(pl)
        elif pl.x < cell.xRight:
            cell.pullRightSection(pl.x)
            return cell, cell.findNodeToConnect(pl)
        elif pl.x > cell.xRight and cell.next is not None:
            return self.__addPayloadToCell(cell.next, pl)
        else:
            raise Exception()

In [26]:
payloads = [Payload(x) for x in (-0.0003, 11, 19.5, 17.9996, 18, 18.5, 3)]
print(f"payloads = {payloads}")

leftNodes = [Node(0, i + 1) for i in range(3)]
rightNodes = [Node(20, i + 1) for i in range(3)]
p = Partition(0, 20, leftNodes, rightNodes, Node(0), AcNetworkLattice(), CircuitGraph())
p.ensureCapacity([p.x for p in payloads])
p.initCells()
p.addPayloads(payloads)

p.arrangePayloads()

cell = p.firstCell
cellSummary = []
while cell is not None:
    cellSummary.append(f"{cell.xLeft}..{cell.xRight}")
    cell = cell.next
print("cells = " + " | ".join(cellSummary))

payloads = [{x: -0.0003, li: 1, br: 0}, {x: 11, li: 1, br: 0}, {x: 19.5, li: 1, br: 0}, {x: 17.9996, li: 1, br: 0}, {x: 18, li: 1, br: 0}, {x: 18.5, li: 1, br: 0}, {x: 3, li: 1, br: 0}]
{3, 11, 18.5, 19.5, 18.0}
cells = 0..3 | 3..11 | 11..18.0 | 18.0..18.5 | 18.5..19.5 | 19.5..20
